<h1> KNeighborsClassifier <h1>

In [422]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import set_config
import warnings

In [423]:
warnings.filterwarnings("ignore", category=FutureWarning)
set_config(display = "diagram")

<h1>Datensatz einlesen<h1>

In [424]:
path = r'./out/df_clean.csv'
df = pd.read_csv(path, sep=';', decimal=',')

In [425]:
df.head()

,ZIP_CODE,DAYS_SINCE_PURCHASE,PURCHASE_VISITS,TOTAL_NET_SALES,CC_CARD,AVE_AMOUNT_SPENT,AMSPEND,PSSPEND,CCSPEND,AXSPEND,TMONSPEND,OMONSPEND,SMONSPEND,PREVPD,GMP,PROMOS,DAYS,FREDAYS,MARKDOWN,CLASSES,COUPONS,STYLES,STORES,VALPHON,WEB,MAILED,RESPONDED,RESPONSERATE,HI,LTFREDAY,PERCRET,RESP,CLUSTYPE_Cluster_0,CLUSTYPE_Cluster_1,CLUSTYPE_Cluster_10,CLUSTYPE_Cluster_11,CLUSTYPE_Cluster_12,CLUSTYPE_Cluster_13,CLUSTYPE_Cluster_14,CLUSTYPE_Cluster_15,CLUSTYPE_Cluster_16,CLUSTYPE_Cluster_17,CLUSTYPE_Cluster_18,CLUSTYPE_Cluster_19,CLUSTYPE_Cluster_2,CLUSTYPE_Cluster_20,CLUSTYPE_Cluster_21,CLUSTYPE_Cluster_22,CLUSTYPE_Cluster_23,CLUSTYPE_Cluster_24,CLUSTYPE_Cluster_25,CLUSTYPE_Cluster_26,CLUSTYPE_Cluster_27,CLUSTYPE_Cluster_28,CLUSTYPE_Cluster_29,CLUSTYPE_Cluster_3,CLUSTYPE_Cluster_30,CLUSTYPE_Cluster_31,CLUSTYPE_Cluster_32,CLUSTYPE_Cluster_33,CLUSTYPE_Cluster_34,CLUSTYPE_Cluster_35,CLUSTYPE_Cluster_36,CLUSTYPE_Cluster_37,CLUSTYPE_Cluster_38,CLUSTYPE_Cluster_39,CLUSTYPE_Cluster_4,CLUSTYPE_Cluster_40,CLUSTYPE_Cluster_41,CLUSTYPE_Cluster_42,CLUSTYPE_Cluster_43,CLUSTYPE_Cluster_44,CLUSTYPE_Cluster_45,CLUSTYPE_Cluster_46,CLUSTYPE_Cluster_47,CLUSTYPE_Cluster_48,CLUSTYPE_Cluster_49,CLUSTYPE_Cluster_5,CLUSTYPE_Cluster_50,CLUSTYPE_Cluster_6,CLUSTYPE_Cluster_7,CLUSTYPE_Cluster_8,CLUSTYPE_Cluster_9
0,1001,208,2,368.46,0,184.23,0.0,0.0,368.46,0.0,0.00,0.00,0.00,0.00,0.60,17,666,333.00,0.08,9,1,11,1,0,0,5,0,0.0000,0.3181,111.00,0.000000e+00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1028,6,4,258.00,1,64.50,0.0,0.0,258.00,0.0,138.00,55.99,258.00,0.00,0.54,14,696,174.00,0.33,6,0,14,1,1,0,4,2,0.5000,0.3272,43.50,3.000000e-08,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1056,327,2,77.00,0,38.50,0.0,0.0,77.00,0.0,0.00,0.00,0.00,39.00,0.62,10,343,171.50,0.11,1,0,2,1,0,0,4,0,0.0000,1.0000,68.60,0.000000e+00,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1118,66,8,846.06,1,105.75,0.0,0.0,846.06,0.0,104.94,0.00,373.87,166.25,0.43,24,701,87.62,0.29,15,3,35,1,1,0,9,6,0.6667,0.2327,26.96,0.000000e+00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1107,49,1,87.44,0,87.44,0.0,0.0,87.44,0.0,87.44,0.00,87.44,0.00,0.22,0,49,49.00,0.42,4,0,8,1,1,0,0,0,0.0000,0.2852,24.50,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h2>Funktionen<h2>

<h3>Funktion für den Train Test Validation Split<h3>

In [426]:
#nimmt Dataframe entgegen und gibt Trainings-, Test- und Validationdataframes im Verhältnins (80%,20%,5%) zurück
def split_train_test(df):
    random_state = 4711
    features = [x for x in df.columns if x!= "RESP"]
    X = df[features]
    y = df["RESP"]
    X_train, X_test_2, y_train, y_test_2 = train_test_split(X, y, test_size = 0.05, random_state=random_state)
    X_train, X_test_1, y_train, y_test_1 = train_test_split(X_train, y_train, test_size = 0.2, random_state=random_state)
    return X_train, X_test_2, y_train, y_test_2, X_test_1, y_test_1

<h3>Funktion für die Errechnung des Gewinns/Verlustes auf Grundlage der Gewinn-/Verlustmatrix<h3>

In [427]:
#nimmt die tatsächlichen und die vorhergesagten Werte als Series entgegen und gibt den auf Grundlage der Aufgabenstellung errechneten Gewinn/Verlust zurück
def get_result_score(y_test, y_predicted):

    TP = 0
    FP = 0
    TN = 0
    FN = 0

    y_true = y_test.to_list()
    y_predicted = y_predicted.tolist()

    for i in range(len(y_predicted)):
        if y_true[i]==y_predicted[i]==1:
            TP += 1
        if y_predicted[i]==1 and y_true[i]!=y_predicted[i]:
            FP += 1
        if y_true[i]==y_predicted[i]==0:
            TN += 1
        if y_predicted[i]==0 and y_true[i]!=y_predicted[i]:
            FN += 1


    return (TP*16+FP*(-2)+TN*(0)+FN*(-16))

In [428]:
# Trainings-,Test- und Vaildationsdaten erstellen
X_train, X_test_2, y_train, y_test_2, X_test_1, y_test_1 = split_train_test(df)

<h2>KNearstNeighbor Classifier<h2>

**Betrachtete Hyperparameter KNearst Neighbor**

*  n_neighbors: Anzahl betrachteter Nachbarn
*  weights: Gewichtung der nächsten Nachbarn [uniform, distance]
*  metric: Metrik für die Distanzberechnung [euclidean, minkowski]

<h3>Feature Selection der 10 wichtigsten Features über ANOVA F-Werte<h3>

In [429]:
# Betrachtung der ANOVA F-Werte statt der Chi Quadrat Werte für die Feature Selection, das negative Werte im Datensatz enthalten sind https://stackoverflow.com/questions/25792012/feature-selection-using-scikit-learn"""
skBest_classif = SelectKBest(f_classif, k=10)
std_slc = preprocessing.StandardScaler()
knn = KNeighborsClassifier()

<h3>Erstellen einer Pipeline mit Standardisierung (z-Values), Feature Selection (ANOVA F-Werte) und KNearstNeighbor Modell<h3>

In [430]:
step_order = [("scaler",std_slc),("skBest_classif",skBest_classif),("knn",knn)]
pipe = Pipeline(step_order)
pipe

Pipeline(steps=[('scaler', StandardScaler()), ('skBest_classif', SelectKBest()),
                ('knn', KNeighborsClassifier())])

In [431]:
#zeigt die auf Grundlage des ANOVA F-Werts wichtigsten features
pipe.fit(X_train,y_train)
features = pipe.named_steps['skBest_classif']
X_train.columns[features.get_support()]

Index(['PURCHASE_VISITS', 'TOTAL_NET_SALES', 'SMONSPEND', 'CLASSES', 'COUPONS',
       'STYLES', 'STORES', 'RESPONDED', 'RESPONSERATE', 'LTFREDAY'],
      dtype='object')

In [432]:
# Listen mit 20 Einträgen ertsellen
mean_acc = np.zeros(20)
mean_score = np.zeros(20)
# Schleife mit 20 Iterationen
for i in range(1,21):
    # Je Iteration erstellen und Trainieren eines Knn Modells mit einer Anzahl nächster Nachbarn von 1-20
    pipe.set_params(knn__n_neighbors=i).fit(X_train, y_train)
    # Vorhersage der Klasse auf Grundlage der Test1 Daten
    y_predicted = pipe.predict(X_test_1)
    # Füllen der Listen mit der so erzielten Accuracy und Gewinn/Verlusts
    mean_acc[i-1] = metrics.accuracy_score(y_test_1, y_predicted)
    mean_score[i-1] = get_result_score(y_test_1, y_predicted)

KeyboardInterrupt: 

In [ ]:
# Darstellung der Accuracy in Abhängigkeit von der Anzahl nächster Nachbarn
loc = np.arange(1,21,step=1.0)
plt.figure(figsize = (15, 6))
plt.plot(range(1,21), mean_acc)
plt.xticks(loc)
plt.xlabel('Number of Neighbors ')
plt.ylabel('Accuracy')
plt.title("Accuracy Score in Abhängigkeit von der Anzahl berücksichtigter Nachbarn")
plt.show()

In [ ]:
# Darstellung des Gewinns/Verlusts in Abhängigkeit von der Anzahl nächster Nachbarn
loc = np.arange(1, 21, step=1.0)
plt.figure(figsize=(15, 6))
plt.plot(range(1, 21), mean_score)
plt.xticks(loc)
plt.xlabel('Number of Neighbors ')
plt.ylabel('Gewinn/Erlös ')
plt.title("Gewinn/Erlös in Abhängigkeit von der Anzahl berücksichtigter Nachbarn")
plt.show()

In [ ]:
# Lösungsraum für die Hyperparameteroptimierung festlegen
grid_params = {"knn__n_neighbors" : [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
              "knn__weights" : ["uniform","distance"],
              "knn__metric": ["euclidean","minkowski"]
              }

#eigene Funktion als Scorer für die GridSearch
result_scorer = make_scorer(get_result_score)

# GirdSearch mit 3 fold cross validation, Suche über die zuvor definierten Parameter

#Suche nach optimalen Parametern unter Berücksichtigung des Gewinns/Erlöses
knn_random_result_score = GridSearchCV(estimator = pipe, param_grid = grid_params,
                                       cv = 3, n_jobs = -1, scoring = result_scorer, error_score="raise")

#Suche nach optimalen Parametern unter Berücksichtigung der Accuracy
knn_random_result_accuracy = GridSearchCV(estimator=pipe,param_grid = grid_params, cv = 3, n_jobs = -1,error_score="raise")

#Suche nach optimalen Parametern unter Berücksichtigung des Recalls
knn_random_result_recall = GridSearchCV(estimator = pipe, param_grid = grid_params,
                                        cv = 3, n_jobs = -1, scoring = "recall", error_score="raise")

In [ ]:
# GridSearch Instanzen mit Trainingsdaten fitten
knn_random_result_score_res = knn_random_result_score.fit(X_train, y_train)
knn_random_result_accuracy_res = knn_random_result_accuracy.fit(X_train, y_train)
knn_random_result_recall_res = knn_random_result_recall.fit(X_train, y_train)

# Ausgabe des jeweils besten Scores und der optimalen dafür genutzten Parameter
print("Bester Score: ",knn_random_result_score_res.best_score_)
print("Mit Parametern: ",knn_random_result_score_res.best_params_)
print("Beste Accuracy: ",knn_random_result_accuracy_res.best_score_)
print("Mit Parametern: ",knn_random_result_accuracy_res.best_params_)
print("Bester Recall: ",knn_random_result_recall_res.best_score_)
print("Mit Parametern: ",knn_random_result_recall_res.best_params_)

In [ ]:
# Funktion die der Threshold Wert und die Wahrscheinlichkeit der Klassenzugehörigkeit 1 entgegennimmt und einen diesen zurückgibt, solange die Wahrscheinlichkeit der Klassenzugehörigkeit über dem Threshold liegt (sonst 0)
def to_labels(probabilities, threshold):
    return(probabilities >= threshold).astype("int")

#Thresholds evaluieren
# Erstellen und Trainieren einer neuen Modellinstanz mit den zuvor ermittelten besten Hyperparametern bei Berücksichtung des Gewinns/Verlusts
pipe.set_params(knn__n_neighbors=2, knn__metric = "euclidiean", knn__weights="uniform").fit(X_train, y_train)
# Vorhersage der Zielvariablen auf für die Test1 Daten
y_predicted = (pipe.predict_proba(X_test_1))
# Ermitteln der Wahrscheinlichkeiten mit denen eine Klassifikation der Klasse 1 (Kunde reagiert) erfolgt
probs = y_predicted[:,1]
# Darstellung der Wahrscheinlichkeiten in einem Histogramm
plt.hist(probs, bins=25, edgecolor='black')
plt.xlabel('Wahrscheinlichkeit der Klassenzugehörigkeit 1')
plt.ylabel('Anzahl Predictions')
plt.title('Wahrscheinlichkeiten der Predictions unter Berücksichtigung der 2 nächsten Nachbarn')
plt.show()

# Liste mit Thresholds von 0 bis 1 in 0.001 Steps
thresholds = np.arange(0,1,0.001)
# Ermitteln der Scores unter Berücksichtigung der unterschiedlichen Thresholds
scores = [get_result_score(y_test_1,to_labels(probs,t)) for t in thresholds]
# Ermitteln des List Indexes an dem der Gewinn/Erlös am größten ist
max_threshold = np.argmax(scores)
# Ausgabe des besten Gewinns/Verlusts mit optimalen Threshold
print("Threshold= ",thresholds[max_threshold],"Score-Wert= ",scores[max_threshold], "€")
# Darstellung des Gewinns/Verlusts unter Berücksichtigung der Thresholds
plt.plot(np.arange(0,1,0.001), scores)
plt.xticks(np.arange(0,1,0.1))
plt.title("Score-Wert in Abhängigkeit vom Classification Threshold")
plt.xlabel("Threshold")
plt.ylabel('Score')
plt.show()

# Erstellen und Trainieren einer neuen Modellinstanz mit den zuvor ermittelten besten Hyperparametern bei Berücksichtung der Accuracy
pipe.set_params(knn__n_neighbors=20, knn_metric = "euclidean", knn__weights="uniform").fit(X_train, y_train)
# Vorhersage der Zielvariablen auf für die Test1 Daten
y_predicted = (pipe.predict_proba(X_test_1))
# Ermitteln der Wahrscheinlichkeiten mit denen eine Klassifiaktion der Klasse 1 (Kunde reagiert) erfolgt
probs = y_predicted[:,1]
# Darstellung der Wahrscheinlichkeiten in einem Histogramm
plt.hist(probs, bins=25, edgecolor='black')
plt.xlabel('Wahrscheinlichkeit der Klassenzugehörigkeit 1')
plt.ylabel('Anzahl Predictions')
plt.title('Wahrscheinlichkeiten der Predictions unter Berücksichtigung der 20 nächsten Nachbarn')
plt.show()

# Liste mit Thresholds von 0 bis 1 in 0.001 Steps
thresholds = np.arange(0,1,0.001)
# Ermitteln der Scores unter Berücksichtigung der unterschiedlichen Thresholds
scores = [get_result_score(y_test_1,to_labels(probs,t)) for t in thresholds]
# Ermitteln des List Indexes an dem der Gewinn/Erlös am größten ist
max_threshold = np.argmax(scores)
# Ausgabe des besten Gewinns/Verlusts mit optimalen Threshold
print("Threshold= ",thresholds[max_threshold],"Score-Wert= ",scores[max_threshold], "€")
# Darstellung des Gewinns/Verlusts unter Berücksichtigung der Thresholds
plt.plot(np.arange(0,1,0.001), scores)
plt.xticks(np.arange(0,1,0.1))
plt.title("Score-Wert in Abhängigkeit vom Classification Threshold")
plt.xlabel("Threshold")
plt.ylabel('Score')
plt.show()

In [ ]:
# Erstellen und Trainieren einer neuen Modellinstanz
pipe.set_params(knn__n_neighbors=2, knn__metric = "euclidiean", knn__weights="uniform").fit(X_train, y_train)
# Vorhersage der Klasse für die finalen Test2 Daten unter Berücksichtigung eines Classification Thresholds von 5%
y_predicted = (pipe.predict_proba(X_test_2)[:,1]>=0.05).astype(bool)
# Ermitteln der Accuracy, Precision, Recall, F1Score und Gewinn/Erlös
accuracy = accuracy_score(y_test_2, y_predicted)
precision = precision_score(y_test_2, y_predicted)
recall = recall_score(y_test_2, y_predicted)
f1score = f1_score(y_test_2, y_predicted)
score = get_result_score(y_test_2, y_predicted)

# Ausgabe der Scores
print("Accuracy: ",accuracy*100, "%")
print("Recall: ", recall*100, "%")
print("Precision: ", precision*100, "%")
print("F1 Score: ", f1score*100, "%")
print("Score: ", score, "€")

# Darstellung der Confusion Matrix
matrix = confusion_matrix(y_test_2,y_predicted,labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=knn.classes_)
disp.plot()
plt.grid(False)
plt.show()

In [ ]:
# Erstellen und Trainieren einer neuen Modellinstanz
pipe.set_params(knn__n_neighbors=17, knn__metric = "euclidiean", knn__weights="uniform").fit(X_train, y_train)
# Vorhersage der Klasse für die finalen Test2 Daten unter Berücksichtigung eines Classification Thresholds von 5%
y_predicted = (pipe.predict_proba(X_test_2)[:,1]>=0.05).astype(bool)
# Ermitteln der Accuracy, Precision, Recall, F1Score und Gewinn/Erlös
accuracy = accuracy_score(y_test_2, y_predicted)
precision = precision_score(y_test_2, y_predicted)
recall = recall_score(y_test_2, y_predicted)
f1score = f1_score(y_test_2, y_predicted)
score = get_result_score(y_test_2, y_predicted)

# Ausgabe der Scores
print("Accuracy: ",accuracy*100, "%")
print("Recall: ", recall*100, "%")
print("Precision: ", precision*100, "%")
print("F1 Score: ", f1score*100, "%")
print("Score: ", score, "€")

# Darstellung der Confusion Matrix
matrix = confusion_matrix(y_test_2,y_predicted,labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=knn.classes_)
disp.plot()
plt.grid(False)
plt.show()

<h2>Baseline Strategie<h2>

In [ ]:
# Errechnung des Gewinn/Verlusts bei Befolgung der Baseline Strategie
len(y_test_2)
count_RESP_yes = sum(y_test_2)
count_RESP_no = len(y_test_2) - sum(y_test_2)
baseline = (count_RESP_yes*16)+(count_RESP_no*(-2))
print(baseline)